This notebook investigates the behavior of the matrix profile in finding the top-10 motifs in the `freezer` dataset.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import requests

We work using the z-normalized euclidean distance, implemented as follows

In [2]:
def znorm(x):
    return (x - x.mean()) / x.std()

def zeucl(x, y):
    return np.linalg.norm(znorm(x) - znorm(y))

First, we download the dataset

In [3]:
def download(url, filename, dtype=np.float64):
    if not os.path.isfile(filename):
        r = requests.get(url)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
    return np.loadtxt(filename, dtype=dtype)

In [4]:
freezer = download('https://figshare.com/ndownloader/files/36982390', 
                   'figshare.txt.gz')

Then, we compute the matrix profile. To do so, we use [SCAMP](https://github.com/zpzim/SCAMP) using the provided Docker image on a GPU-enabled machine, with the following command:

```
singularity run --nv \
    docker://zpzim/scamp:latest /SCAMP/build/SCAMP \
    --window 5000 --input_a_file_name=freezer.txt
```

For convenience, the output has been saved in a couple of files and also uploaded to figshare.

In [5]:
mp = download('https://figshare.com/ndownloader/files/39415483', 
              'freezer.mp.5000.txt')
mp_index = download('https://figshare.com/ndownloader/files/39415486', 
                    'freezer.mp.index.5000.txt', dtype=np.int32)

With the following function we iterate through the motifs discovered by the matrix profile so that no overlap between found subsequences occurs. The procedure is as follows. First we find the minimum in the matrix profile, take note of the corresponding indices and distance, and mask out all the subsequences that overlap with the two that have just been found, which are returned.
Then, we find the next minimum in the matrix profile, such that both corresponding subsequence do not overlap with any previously defined subsequence.

In [6]:
def iter_motifs(mp, mp_index, w):
    """Enumerate the motifs, excluding overlapping subsequences."""
    def mask_out(mask, i):
        start = max(0, i-w)
        end = min(len(mask), i+w)
        mask[start:end] = False
        
    mask = np.ones_like(mp, dtype=bool)
    mp_sorted = np.argsort(mp, axis=0)
    for i in mp_sorted:
        j = mp_index[i]
        dist = mp[i]
        if mask[i] and mask[j]:
            # mask out overlapping subsequences
            mask_out(mask, i)
            mask_out(mask, j)
            # return motif pair
            yield (i, j, dist)
            
for rank, motif in enumerate(iter_motifs(mp, mp_index, 5000)):
    if rank > 10:
        break
    print(rank, motif)


0 (3705031, 1834102, 4.195242485)
1 (3698075, 4733298, 5.765751866)
2 (2352371, 4186995, 7.077046765)
3 (4002563, 3993450, 7.318316307)
4 (4618976, 4812738, 9.207241828)
5 (1825969, 1993859, 9.366285725)
6 (1697587, 1408089, 10.56533893)
7 (5169982, 6429402, 11.46242184)
8 (6641806, 5230708, 12.46052708)
9 (6339277, 191377, 12.50717434)
10 (36578, 3988437, 12.73866733)


The above list reports the first 10 motifs found in such way.

Consider however the following pair of subsequences (which is the one mentioned in the paper).
Given their distance, this pair should be the rank-7 motif in the list above. However, this particular motif with this particular window length _with this_ definition of trivial match (no overlapping can happen at all) is not found in the matrix profile.

In [7]:
w = 5000
i = 3815625
j = 5170040

zeucl(freezer[i:i+w], freezer[j:j+w])

11.337788263102953

Why is the above motif not found in the matrix profile?

Let's first look at the subsequence at `i`. Its nearest neighbor is at distance 10.33, and is the subsequence 4732333, which is only 965 timestamps away (with a window length of 5000) from subsequence 4733298, which participates in the motif of rank 2 and therefore cannot be re-used to form another motif.

In [8]:
mp[i], mp_index[i]

(10.33695558, 4732333)

Similarly, the subsequence at index `j` has its nearest neighbor at subsequence `1825878`, which is only 91 timestamps away from subsequence `1825969`.

In [9]:
mp[j], mp_index[j]

(11.02213133, 1825878)

Therefore, we have a pair of subsequences that are not the nearest neighbor of each other, and that nonetheless should part of the top-10 nearest neighbors, as they are defined in our paper.

Given that the matrix profile contains information only about the nearest neighbors, it does not contain the information needed to correctly place this pair in the ranking of motifs.